In [304]:
import findspark
findspark.init('/usr/local/spark')

In [308]:
from pyspark import SparkContext, SparkConf
import pandas as pd
import re
from operator import add
import numpy as np
import os

In [309]:
questionOriginalFile = pd.read_csv("_Questions.csv",encoding="latin-1", iterator = True, chunksize=100)
hasHeader = False

os.remove("Questions.csv")
for chunk in questionOriginalFile:
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace("\n", " "))
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace(",", ";"))
    if hasHeader:
        chunk.to_csv("Questions.csv", mode = "a", encoding="utf-8", header=False, index = False)
    else:
        chunk.to_csv("Questions.csv", mode = "a", encoding="utf-8", header=True, index = False)
print("finish processing Questions.csv")

finish processing Questions.csv


In [310]:
answerOriginalFile = pd.read_csv("_Answers.csv",encoding="latin-1", iterator = True, chunksize=100)
hasHeader = False

os.remove("Answers.csv")
for chunk in answerOriginalFile:
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace("\n", " "))
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace(",", ";"))
    if hasHeader:
        chunk.to_csv("Answers.csv", mode = "a", encoding="utf-8", header=False, index = False)
    else:
        chunk.to_csv("Answers.csv", mode = "a", encoding="utf-8", header=True, index = False)
print("finish processing Answers.csv")

finish processing Answers.csv


In [311]:
sc = SparkContext.getOrCreate()

In [312]:
# questionFile=sc.textFile("Questions.csv")
questionFile=sc.textFile("test_q.csv")
# answerFile=sc.textFile("Answers.csv")
answerFile=sc.textFile("test_a.csv")
# tagFile = sc.textFile("Tags.csv")
tagFile = sc.textFile("test_t.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finish reading files")

finish reading files


In [313]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first()
question = questionWithHeader.filter(lambda x: x != header)

In [314]:
stopWordsList = stopWordsFile.collect()

In [315]:
questionLower = question.map(lambda x: (x[0], (x[5]+x[6]).lower()))

In [316]:
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
print(questionLower.count())

106


In [317]:
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.count())

7996


In [318]:
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
print(questionPair.count())

3998


In [319]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
print(questionPairfilter.count())

3889


In [320]:
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
print(questionPairStage1.count())

1919


In [321]:
print(questionPairStage1.count())

1919


In [322]:
N = questionLower.count()
print(N)

106


In [323]:
print(questionPairStage1.take(5))

[('80@execute', 4), ('80@multiple', 2), ('80@statement', 2), ('80@database', 1), ('80@generation', 1)]


In [324]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage2MapPhase.take(5))

[('execute', '80=4'), ('multiple', '80=2'), ('statement', '80=2'), ('database', '80=1'), ('generation', '80=1')]


In [325]:
stage2Map = questionPairStage2MapPhase.countByKey()

In [326]:
print(stage2Map.get('execute'))

1


In [327]:
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.take(5))

[('execute@80', 11.128338413705677), ('multiple@80', 5.548688364952579), ('statement@80', 6.035776454821954), ('database@80', 2.583997552432231), ('generation@80', 4.663439094112067)]


In [328]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage3MapPhase.take(5))

[('80', 'execute=11.128338413705677'), ('80', 'multiple=5.548688364952579'), ('80', 'statement=6.035776454821954'), ('80', 'database=2.583997552432231'), ('80', 'generation=4.663439094112067')]


In [329]:
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.take(5))

[('180', 229.7145562811237), ('330', 1341.3598263114907), ('470', 448.5714876428375), ('580', 2123.868076874934), ('930', 170.42691150448204)]


In [330]:
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

22.038131748297413


In [331]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
print(questionPairStage3.take(10))

[('80', 'execute=0.18639896556738128'), ('80', 'multiple=0.09294017965962616'), ('80', 'statement=0.10109887440061487'), ('80', 'database=0.04328179579880556'), ('80', 'generation=0.07811231028509996'), ('80', 'http=0.061783416134000416'), ('80', 'en=0.0770694152873754'), ('80', 'wikipedia=0.10109887440061487'), ('80', 'wiki=0.10109887440061487'), ('80', 'adobe=0.13956220568671052')]


In [332]:
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

[('180', ['wheels=0.3076890572649864', 'pseudo=0.3076890572649864', 'solved=0.3076890572649864', 'stuck=0.3076890572649864', 'distinguishable=0.3076890572649864', 'parameter=0.2619558980603028', 'g=0']), ('330', ['classes=0.3193517411118351', 'video=0.227500315486462', 'workhorse=0.2155898812650258', 'book=0.2155898812650258', 'mine=0.18354582207600814', 'pause=0.12733085684478265', 'encoding=0.12733085684478265']), ('470', ['homegrown=0.3728079699966661', 'services=0.284981923834969', 'ready=0.22018639269940712', 'numerous=0.22018639269940712', 'opposed=0.22018639269940712', 'auto=0.22018639269940712', 'creates=0.22018639269940712']), ('580', ['live=0.2825013570494739', 'products=0.17133151385886897', 'scripts=0.17133151385886897', 'enterprise=0.17133151385886897', 'manager=0.17133151385886897', 'express=0.17133151385886897', 'gate=0.17133151385886897']), ('930', ['connect=0.5149298451290975', 'simplest=0.3572211977022691', 'loop=0.3572211977022691', 'recordset=0.3572211977022691', 'q

In [333]:
outMap=questionPairStage4.collectAsMap()

In [335]:
# mock expected keywords
keywords=list(set(questionPairStage4.map(lambda x: list(map((lambda x: x.split("=")[0]), x[1]))).reduce(add)))
print(keywords)

['organization', 'ul', 'confirm', 'big', 'sqlstatement', 'wheel', 'languages', 'event', 'manager', 'syntax', 'providers', 'stacks', 'matching', 'objects', 'site', 'programming', 'best', 'ux', 'simple', 'create', 'explaining', 'dns', 'payload', 'times', 'heard', 'update', 'parameter', 'autocomplete', 'dataset', 'bit', 'updating', 'querying', 'simplest', 'ends', 'structure', 'homegrown', 'stuck', 'sqlconnection', 'cmdlets', 'replace', 'numerous', 'rspec', 'supports', 'stored', 'respective', 'bazaar', 'query', 'mysql', 'studio', 'terms', 'peak', 'hkey', 'verifications', 'resolvepath', 'trim', 'ado', 'icard', 'version', 'purchased', 'catching', 'winforms', 'cards', 'property', 'aix', 'c', 'values', 'bean', 'messy', 'br', 'today', 'questions', 'hashmaps', 'attempt', 'table', 'dim', 'gt', 'wheels', 'moment', 'datagrid', 'embedding', 'recordset', 'connect', 'net', 'simply', 'pm', 'choice', 'video', 'ruby', 'liba', 'playing', 'array', 'customusername', 'core', 'subversion', 'solo', 'cms', 'tra

In [336]:
newQuestion = sc.wholeTextFiles("new_q.txt")
newQuestion = newQuestion.map(lambda x: re.sub('\n', ' ', x[1]))
newQuestion = newQuestion.map(lambda x: re.sub('<.*?>', ' ', x))
newQuestion = newQuestion.map(lambda x: re.sub('[^a-zA-Z\']+', ' ', x))

print(newQuestion.take(1))

[" I've written a database generation script in SQL and want to execute it in my Adobe AIR application Create Table tRole roleID integer Primary Key roleName varchar Create Table tFile fileID integer Primary Key fileName varchar fileDescription varchar thumbnailID integer fileFormatID integer categoryID integer isFavorite boolean dateAdded date globalAccessCount integer lastAccessTime date downloadComplete boolean isNew boolean isSpotlight boolean duration varchar Create Table tCategory categoryID integer Primary Key categoryName varchar parent categoryID integer I execute this in Adobe AIR using the following methods public static function RunSqlFromFile fileName String void var file File File applicationDirectory resolvePath fileName var stream FileStream new FileStream stream open file FileMode READ var strSql String stream readUTFBytes stream bytesAvailable NonQuery strSql public static function NonQuery strSQL String void var sqlConnection SQLConnection new SQLConnection sqlConnec

In [337]:
newQuestionKeywordsStageOne = newQuestion.flatMap(lambda x: x.lower().split(" ")).filter(lambda x: x != '' and x not in stopWordsList).map(lambda x: (x, 1)).reduceByKey(add).map(lambda x: (x[0], 'new=' + str(x[1])))
print(newQuestionKeywordsStageOne.take(10))


[('written', 'new=1'), ('database', 'new=1'), ('generation', 'new=1'), ('script', 'new=1'), ('sql', 'new=1'), ('execute', 'new=3'), ('adobe', 'new=2'), ('air', 'new=2'), ('application', 'new=1'), ('create', 'new=3')]


In [338]:
newQuestionKeywordsStageTwoMap = newQuestionKeywordsStageOne.union(questionPairStage2MapPhase).countByKey()

newQuestionKeywordsStageTwo = newQuestionKeywordsStageOne.map(lambda x: (x[0], (1 + np.log(int(x[1].split("=")[1]))) * np.log((N + 1)/newQuestionKeywordsStageTwoMap.get(x[0]))))
print(newQuestionKeywordsStageTwo.take(10))

[('written', 3.574216545793796), ('database', 2.4756042571256867), ('generation', 3.979681653901961), ('script', 3.0633909220278057), ('sql', 2.033771504846648), ('execute', 8.351808823865683), ('adobe', 6.051674667221473), ('air', 6.738186771830246), ('application', 2.4756042571256867), ('create', 5.442514441908174)]


In [339]:
newQuestionKeywordsStageThreeSquare = newQuestionKeywordsStageTwo.map(lambda x: np.square(x[1])).reduce(add)

newQuestionKeywordsStageThree = newQuestionKeywordsStageTwo.map(lambda x: (x[0], x[1]/np.sqrt(newQuestionKeywordsStageThreeSquare)))
print(newQuestionKeywordsStageThree.take(50))

[('written', 0.07247168990263128), ('database', 0.050195958119879744), ('generation', 0.08069299972106751), ('script', 0.06211406446903707), ('sql', 0.04123724904287243), ('execute', 0.16934332082412465), ('adobe', 0.12270523742905481), ('air', 0.13662512496864992), ('application', 0.050195958119879744), ('create', 0.11035375553524643), ('table', 0.11035375553524643), ('trole', 0.13662512496864992), ('roleid', 0.08069299972106751), ('integer', 0.22317233248190185), ('primary', 0.16934332082412465), ('key', 0.13984853817968554), ('rolename', 0.08069299972106751), ('varchar', 0.18911037521009366), ('tfile', 0.08069299972106751), ('fileid', 0.08069299972106751), ('filename', 0.16934332082412465), ('filedescription', 0.08069299972106751), ('thumbnailid', 0.08069299972106751), ('fileformatid', 0.08069299972106751), ('categoryid', 0.16934332082412465), ('isfavorite', 0.08069299972106751), ('boolean', 0.19255725021623232), ('dateadded', 0.08069299972106751), ('globalaccesscount', 0.0806929997

In [340]:
newQuestionKeywordsStageFour = newQuestionKeywordsStageThree.filter(lambda x: x[0] in keywords).sortBy(lambda x: -x[1])
print(newQuestionKeywordsStageFour.take(50))

[('sqlconnection', 0.22527544607170702), ('sqlstatement', 0.22527544607170702), ('stream', 0.19255725021623232), ('strsql', 0.19255725021623232), ('varchar', 0.18911037521009366), ('void', 0.13662512496864992), ('resolvepath', 0.13662512496864992), ('nonquery', 0.13662512496864992), ('string', 0.12259519636576749), ('create', 0.11035375553524643), ('table', 0.11035375553524643), ('query', 0.09890903250577458), ('queries', 0.058417267938315975), ('database', 0.050195958119879744), ('sql', 0.04123724904287243)]


In [342]:
# mock expected questions inside buckets <Qi, sum(Ai)>
candidateSimilarQuestions = newQuestionKeywordsStageFour.map(lambda x: x[0])
print(candidateSimilarQuestions.take(5))

['sqlconnection', 'sqlstatement', 'stream', 'strsql', 'varchar']


In [187]:
from hashlib import sha1
from datasketch.minhash import MinHash